In [106]:
import pandas as pd
import catboost
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from catboost import CatBoostRegressor
from catboost import Pool, metrics, cv
from sklearn import model_selection


In [107]:
df = pd.read_csv("Train.csv")

In [108]:
# Convert % to numeric
df['host_response_rate'] = df['host_response_rate'].str.rstrip('%').apply(pd.to_numeric, errors='coerce')
df['host_acceptance_rate'] = df['host_acceptance_rate'].str.rstrip('%').apply(pd.to_numeric, errors='coerce')

In [109]:
# Convert $ to numeric
df["price"] = df["price"].replace('[\$,]', '', regex=True).astype(float)
df["cleaning_fee"] = df["cleaning_fee"].replace('[\$,]', '', regex=True).astype(float)

In [110]:
df = df.dropna()

In [111]:
df = df.drop('reviews_per_month', axis=1)

In [112]:
df1 = df.iloc[:300]
X_val = df1.drop('price', axis=1)

df = df.iloc[300:]

In [113]:
X, y = df.iloc[:, df.columns != 'price'], \
       df.iloc[:, df.columns == 'price']

In [114]:
X_train, X_test, y_train, y_test = \
    model_selection.train_test_split(X, y, test_size=0.3, random_state=0)

In [115]:
categorical_features_indices = np.where(X.dtypes != np.float)[0]

`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


In [116]:
model = CatBoostRegressor(iterations=100,
                          learning_rate=0.001,
                          depth=10,
                          l2_leaf_reg=100, cat_features = categorical_features_indices)
model.fit(X_train, y_train)

0:	learn: 82.1948927	total: 2.39ms	remaining: 237ms
1:	learn: 82.1840907	total: 5.9ms	remaining: 289ms
2:	learn: 82.1695178	total: 6.53ms	remaining: 211ms
3:	learn: 82.1595344	total: 7.64ms	remaining: 183ms
4:	learn: 82.1468030	total: 8.22ms	remaining: 156ms
5:	learn: 82.1412348	total: 12.2ms	remaining: 191ms
6:	learn: 82.1251686	total: 13.2ms	remaining: 175ms
7:	learn: 82.1091604	total: 13.6ms	remaining: 157ms
8:	learn: 82.1012389	total: 19ms	remaining: 192ms
9:	learn: 82.0862778	total: 19.6ms	remaining: 176ms
10:	learn: 82.0685908	total: 19.9ms	remaining: 161ms
11:	learn: 82.0515628	total: 20.3ms	remaining: 149ms
12:	learn: 82.0429892	total: 23.5ms	remaining: 157ms
13:	learn: 82.0357939	total: 25.1ms	remaining: 154ms
14:	learn: 82.0310136	total: 26.6ms	remaining: 151ms
15:	learn: 82.0249586	total: 29.9ms	remaining: 157ms
16:	learn: 82.0122211	total: 30.5ms	remaining: 149ms
17:	learn: 82.0049688	total: 33.9ms	remaining: 155ms
18:	learn: 82.0008756	total: 38.9ms	remaining: 166ms
19:	le

In [117]:
preds = model.predict(X_test)

In [118]:
print('RMSE: ', np.sqrt(mean_squared_error(preds, y_test)))

print('MAE: ', mean_absolute_error(preds, y_test))

RMSE:  101.49724603633601
MAE:  66.33978092021303


In [119]:
train_pool = Pool(X_train, y_train, cat_features = categorical_features_indices)
validate_pool = Pool(X_test, y_test, cat_features = categorical_features_indices)
feature_importances = model.get_feature_importance(train_pool)
feature_names = X_val.columns
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score))

bedrooms: 51.29670327755933
cleaning_fee: 16.89388206794876
room_type: 12.338696309183106
beds: 5.651235267880997
bathrooms: 3.4640506863044256
accommodates: 3.358873695407506
property_type: 1.2426429357174902
host_response_rate: 0.7524340383334068
neighbourhood_cleansed: 0.5576141588449436
review_scores_location: 0.47427830174421
host_neighbourhood: 0.37409840780775816
host_location: 0.37369779354763083
review_scores_cleanliness: 0.35542464300448556
zipcode: 0.30997208993784736
maximum_nights: 0.30361372792125463
minimum_nights: 0.27405909108454723
host_acceptance_rate: 0.2717243141798005
instant_bookable: 0.21585882934191866
guests_included: 0.21048947531382833
review_scores_accuracy: 0.20549295334095025
calculated_host_listings_count: 0.1984150316553039
review_scores_communication: 0.1907545611275521
review_scores_value: 0.158777020246356
smart_location: 0.1416903860454448
review_scores_checkin: 0.12778402599187227
bed_type: 0.11325062679241138
city: 0.05889416960672668
review_score

In [120]:
val = Pool(X_val, feature_names=categorical_features_indices)

CatBoostError: Invalid feature_names type=<class 'numpy.ndarray'> : must be list